# Семинар по рекуррентным нейронным сетям
На этом семинаре мы обучим несколько рекуррентных архитектур для решения задачи сентимент-анализа, то есть предсказания метки тональности предложения.

В общем случае рекуррентная нейронная сеть предназначена для обработки последовательности произвольной длины. Однако при реализации метода оказывается проще зафиксировать длину последовательности (даже в pytorch с их динамическими графами :) Мы так и поступим.

При выполнении задания вы обучите LSTM с разным уровнем "коробочности", а также познакомитесь с различными способами применения дропаута к рекуррентным архитектурам. В рекуррентных архитектурах вариантов, куда можно наложить бинарную маску шума, гораздо больше, чем в нейросетях прямого прохода.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоемкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

### Гиперпараметры

In [2]:
vocab_size = 20000 
index_from = 3
n_hidden = 32 # 128
n_emb = 32 # 128
seq_len = 32 # 200
# small network on small data for seminar purposes
# after # normal size

batch_size = 128
learning_rate = 0.001
num_epochs = 30

### Загрузка данных
Функция load_matrix_imdb скачивает матричные данные, перемешивает и загружает их в numpy-массивы.

Если у вас не установлен wget, скачайте [архив imdb.npz](https://s3.amazonaws.com/text-datasets/imdb.npz)

In [3]:
#from rnn_utils import load_matrix_imdb
import numpy as np
import torch
import torch.utils.data
import os
import re
from collections import defaultdict
import operator

In [4]:
def load_matrix_imdb(path='imdb.npz', num_words=None, skip_top=0,
              maxlen=None, seed=113,
              start_char=1, oov_char=2, index_from=3, **kwargs):
    """
    Modified code from Keras
    Loads data matrixes from npz file, crops and pads seqs and returns
    shuffled (x_train, y_train), (x_test, y_test)
    """
    if not os.path.exists(path):
        print("Downloading matrix data into current folder")
        os.system("wget https://s3.amazonaws.com/text-datasets/imdb.npz")
        
    with np.load(path, allow_pickle=True) as f:
        x_train, labels_train = f['x_train'], f['y_train']
        x_test, labels_test = f['x_test'], f['y_test']

    np.random.seed(seed)
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    x_train = x_train[indices]
    labels_train = labels_train[indices]

    indices = np.arange(len(x_test))
    np.random.shuffle(indices)
    x_test = x_test[indices]
    labels_test = labels_test[indices]

    xs = np.concatenate([x_train, x_test])
    labels = np.concatenate([labels_train, labels_test])

    if start_char is not None:
        xs = [[start_char] + [w + index_from for w in x] for x in xs]
    elif index_from:
        xs = [[w + index_from for w in x] for x in xs]

    if not num_words:
        num_words = max([max(x) for x in xs])
    if not maxlen:
        maxlen = max([len(x) for x in xs])

    # by convention, use 2 as OOV word
    # reserve 'index_from' (=3 by default) characters:
    # 0 (padding), 1 (start), 2 (OOV)
    xs_new = []
    for x in xs:
        x = x[:maxlen] # crop long sequences
        if oov_char is not None: # replace rare or frequent symbols 
            x = [w if (skip_top <= w < num_words) else oov_char for w in x]
        else: # or filter rare and frequent symbols
            x = [w for w in x if skip_top <= w < num_words]
        x_padded = np.zeros(maxlen)#, dtype = 'int32')
        x_padded[-len(x):] = x
        xs_new.append(x_padded)    
            
    idx = len(x_train)
    x_train, y_train = np.array(xs_new[:idx]), np.array(labels[:idx])
    x_test, y_test = np.array(xs_new[idx:]), np.array(labels[idx:])

    return (x_train, y_train), (x_test, y_test)

In [5]:
np.random.seed(0)
(X_train, y_train), (X_test, y_test) = load_matrix_imdb(num_words=vocab_size,
                                                        maxlen=seq_len)

In [6]:
set(y_train) # binary classification

{0, 1}

In [7]:
X_train.shape, X_test.shape

((25000, 32), (25000, 32))

In [8]:
X_train[0] # sequence of coded words

array([1.000e+00, 1.400e+01, 2.200e+01, 1.600e+01, 4.300e+01, 5.300e+02,
       9.730e+02, 1.622e+03, 1.385e+03, 6.500e+01, 4.580e+02, 4.468e+03,
       6.600e+01, 3.941e+03, 4.000e+00, 1.730e+02, 3.600e+01, 2.560e+02,
       5.000e+00, 2.500e+01, 1.000e+02, 4.300e+01, 8.380e+02, 1.120e+02,
       5.000e+01, 6.700e+02, 2.000e+00, 9.000e+00, 3.500e+01, 4.800e+02,
       2.840e+02, 5.000e+00])

In [9]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [10]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long), 
                               torch.tensor(y_train, dtype=torch.long))

In [11]:
test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long), 
                               torch.tensor(y_test, dtype=torch.long))

In [12]:
train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

In [13]:
test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

### Сборка и обучение RNN в pytorch

In [14]:
import os
import torch.optim as optim
import torch.nn as nn

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандарную рекуррентную архитектуру для сентимент-анализа: слой представлений, слой LSTM и полносвязный слой, предсказывающий выход по последнему скрытому состоянию.

Ниже дан код для сборки и обучения нашей нейросети. Обратите внимание на ### pay attention here, указывающие на особенности кода при использовании рекуррентных слоев. 

Чтобы потом было удобно проводить сравнение времени работы разных моделей, запустите код ниже и измерьте время обучения на вашей системе.

In [16]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, rec_layer=nn.LSTM, embedding=nn.Embedding, \
                 dropout=None):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = embedding(vocab_size, embedding_dim)
        if dropout:
            self.rnn = rec_layer(embedding_dim, hidden_dim, dropout=dropout)
        else:
            self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn(embedding) # pay attention here!
        res = self.hidden2label(out[-1])
        return torch.sigmoid(res)
    

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

In [17]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=nn.LSTM,
                       dropout=None).to(device)

In [18]:
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum') + 

In [19]:
def train_epoch(train_loader, model, lossfun, optimizer, device):
    model.train()
    for it, traindata in enumerate(train_loader):
        train_inputs, train_labels = traindata
        train_inputs = train_inputs.to(device) 
        train_labels = train_labels.to(device)
        train_labels = torch.squeeze(train_labels)

        model.zero_grad()        
        output = model(train_inputs.t()) # pay attention here!

        loss = lossfun(output.view(-1), train_labels.float())
        loss.backward()
        optimizer.step()

def evaluate(loader, model, lossfun, device):
    model.eval()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    for it, data in enumerate(loader):
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        output = model(inputs.t()) # pay attention here!
        loss = lossfun(output.view(-1), labels.float()) + torch.norm(WW^T - I)
        total_loss += loss.item()

        # calc testing acc        
        pred = output.view(-1) > 0.5
        correct = pred == labels.byte()
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total
    

def train(train_loader, test_loader, model, lossfun, optimizer, \
          device, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    for epoch in range(num_epochs):
        train_epoch(train_loader, model, lossfun, optimizer, device)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, device)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, device)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [19]:
%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 83.852, Testing Loss: 85.257, Training Acc: 0.619, Testing Acc: 0.598
Epoch:   2/ 30 Training Loss: 75.851, Testing Loss: 80.369, Training Acc: 0.682, Testing Acc: 0.645
Epoch:   3/ 30 Training Loss: 69.847, Testing Loss: 77.532, Training Acc: 0.730, Testing Acc: 0.665
Epoch:   4/ 30 Training Loss: 65.427, Testing Loss: 75.966, Training Acc: 0.752, Testing Acc: 0.676
Epoch:   5/ 30 Training Loss: 61.612, Testing Loss: 77.338, Training Acc: 0.770, Testing Acc: 0.686
Epoch:   6/ 30 Training Loss: 58.236, Testing Loss: 76.019, Training Acc: 0.784, Testing Acc: 0.693
Epoch:   7/ 30 Training Loss: 52.706, Testing Loss: 73.616, Training Acc: 0.815, Testing Acc: 0.702
Epoch:   8/ 30 Training Loss: 48.887, Testing Loss: 76.414, Training Acc: 0.829, Testing Acc: 0.704
Epoch:   9/ 30 Training Loss: 46.773, Testing Loss: 75.653, Training Acc: 0.844, Testing Acc: 0.705
Epoch:  10/ 30 Training Loss: 42.157, Testing Loss: 80.193, Training Acc: 0.860, Testing Acc: 0.706


Нерегуляризованные LSTM часто быстро переобучаются (и мы это видим по точности на контроле). Чтобы с этим бороться, часто используют L2-регуляризацию и дропаут.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. По [ссылке](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) доступен хороший обзор дропаутов для RNN.

Мы реализуем два варианта дропаута для RNN (и третий дополнительно). Заодно увидим, что для реализации различных усовершенствований рекуррентной архитектуры приходится "вскрывать" слой до различной "глубины".

### Реализация дропаута по статье Гала и Гарамани
Начнем с дропаута, описанного в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется перейти от использования слоя nn.LSTM, полностью скрывающего от нас рекуррентную логику, к использованию слоя nn.LSTMCell, обрабатывающего лишь один временной шаг нашей последовательности (а всю логику вокруг придется реализовать самостоятельно). 

Допишите класс RNNLayer. При dropout=0 ваш класс должен работать как обычный слой LSTM, а при dropout > 0 накладывать бинарную маску на входной и скрытый вектор на каждом временном шаге, причем эта маска должна быть одинаковой во все моменты времени.

Дропаут Гала и Гарамани в виде формул (m обознаает маску дропаута):
$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$
(далее обычный шаг рекуррентной архитектуры, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$

In [20]:
def init_h0_c0(num_objects, hidden_size, some_existing_tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    ### your code here
    h0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    c0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    return h0, c0

In [21]:
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)
    
    return dropout masks of size input_size, hidden_size if p is not None
    return one masks if p is None
    """
    ### your code here
    mask0 = some_existing_tensor.new_ones(input_size)
    mask1 = some_existing_tensor.new_ones(hidden_size)
    if p:
        if is_training:
            mask0 *= p
            mask0 = torch.bernoulli(mask0)
            mask1 *= p
            mask1 = torch.bernoulli(mask1)
        else:
            mask0 *= (1-p)
            #mask0 = torch.bernoulli_(mask0)
            mask1 *= (1-p)
            #mask1 = torch.bernoulli_(mask1)
    return mask0, mask1

In [22]:
class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=None):
        super(RNNLayer, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.dropout = dropout
        self.rnn_cell = nn.LSTMCell(input_size, hidden_size)
        
    def forward(self, inp):
        # initialize h_0, c_0
        h_0, c_0 = init_h0_c0(inp.shape[1], self.hidden_size, inp)
        
        # gen masks
        input_mask, hidden_mask = gen_dropout_mask(self.input_size, \
                                                   self.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   inp)
        
        
        ### your code here
        ### implement recurrent logic and return what nn.LSTM returns
        ### do not forget to apply generated dropout masks!
        out = []
        h, c = h_0, c_0
        for i in range(inp.shape[0]):
            h, c = self.rnn_cell(inp[i]*input_mask, (h*hidden_mask, c))
            out.append(h)
        return out, (h, c)

Протестируйте реализованную модель с выключенным дропаутом (слой RNNLayer надо передать в RNNClassifier в качестве rec_layer). Замерьте время обучения (%time). Сильно ли оно увеличилось по сравнению с nn.LSTM (LSTM "из коробки")?

In [23]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=None).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 84.288, Testing Loss: 85.622, Training Acc: 0.613, Testing Acc: 0.589
Epoch:   2/ 30 Training Loss: 76.006, Testing Loss: 80.819, Training Acc: 0.685, Testing Acc: 0.643
Epoch:   3/ 30 Training Loss: 70.186, Testing Loss: 76.887, Training Acc: 0.729, Testing Acc: 0.667
Epoch:   4/ 30 Training Loss: 64.336, Testing Loss: 75.524, Training Acc: 0.755, Testing Acc: 0.684
Epoch:   5/ 30 Training Loss: 60.038, Testing Loss: 74.052, Training Acc: 0.781, Testing Acc: 0.692
Epoch:   6/ 30 Training Loss: 55.509, Testing Loss: 74.238, Training Acc: 0.798, Testing Acc: 0.703
Epoch:   7/ 30 Training Loss: 53.868, Testing Loss: 76.738, Training Acc: 0.807, Testing Acc: 0.703
Epoch:   8/ 30 Training Loss: 48.496, Testing Loss: 75.073, Training Acc: 0.831, Testing Acc: 0.709
Epoch:   9/ 30 Training Loss: 46.116, Testing Loss: 77.050, Training Acc: 0.842, Testing Acc: 0.708
Epoch:  10/ 30 Training Loss: 42.689, Testing Loss: 77.346, Training Acc: 0.858, Testing Acc: 0.711


Протестируйте полученную модель c dropout=0.5, вновь замерив время обучения. Получилось ли побороть переобучение? Сильно ли дольше обучается данная модель по сравнению с предыдущей? (доп. время тратится на генерацию масок дропаута).

In [24]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=0.5).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 88.213, Testing Loss: 88.323, Training Acc: 0.520, Testing Acc: 0.518
Epoch:   2/ 30 Training Loss: 87.817, Testing Loss: 88.098, Training Acc: 0.552, Testing Acc: 0.532
Epoch:   3/ 30 Training Loss: 86.641, Testing Loss: 87.347, Training Acc: 0.584, Testing Acc: 0.556
Epoch:   4/ 30 Training Loss: 84.679, Testing Loss: 86.047, Training Acc: 0.614, Testing Acc: 0.580
Epoch:   5/ 30 Training Loss: 83.256, Testing Loss: 85.047, Training Acc: 0.629, Testing Acc: 0.598
Epoch:   6/ 30 Training Loss: 81.011, Testing Loss: 83.550, Training Acc: 0.652, Testing Acc: 0.613
Epoch:   7/ 30 Training Loss: 79.690, Testing Loss: 82.667, Training Acc: 0.668, Testing Acc: 0.627
Epoch:   8/ 30 Training Loss: 77.882, Testing Loss: 81.628, Training Acc: 0.683, Testing Acc: 0.636
Epoch:   9/ 30 Training Loss: 75.990, Testing Loss: 80.691, Training Acc: 0.692, Testing Acc: 0.646
Epoch:  10/ 30 Training Loss: 74.648, Testing Loss: 79.660, Training Acc: 0.708, Testing Acc: 0.657


In [36]:
model

RNNClassifier(
  (word_embeddings): Embedding(20000, 32)
  (rnn): FastRNNLayer(
    (module): LSTM(32, 32)
  )
  (hidden2label): Linear(in_features=32, out_features=1, bias=True)
)

### Реализация дропаута по статье Гала и Гарамани. Дубль 2

<начало взлома pytorch>

При разворачивании цикла по времени средствами python обучение рекуррентной нейросети сильно замедляется. Однако для реализации дропаута Гала и Гарамани необязательно явно задавать в коде домножение нейронов на маски. Можно схитрить и обойтись использованием слоя nn.LSTM: перед вызовом forward слоя nn.LSTM подменять его веса на веса, домноженные по строкам на маски. А обучаемые веса хранить отдельно. Именно так этот дропаут реализован в библиотеке fastai, код из которой использован в ячейке ниже.

Такой слой реализуется в виде обертки над nn.LSTM. Допишите класс:

In [25]:
import warnings

In [30]:
class FastRNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(FastRNNLayer, self).__init__()
        self.module = nn.LSTM(input_size, hidden_size)
        self.dropout = dropout
        self.layer_names = ['weight_hh_l0', 'weight_ih_l0']
        for layer in self.layer_names:
            # Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            
    def _setweights(self):
        "Apply dropout to the raw weights."
        ### your code here
        ### generate input_mask and hidden_mask (use function gen_dropout_mask)
        input_mask, hidden_mask = gen_dropout_mask(self.module.input_size, \
                                                   self.module.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   self.module.weight_hh_l0.data)
        for layer, mask in zip(self.layer_names, (hidden_mask, input_mask)):
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer].data = raw_w * mask

    def forward(self, *args):
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.filterwarnings("ignore")
            
            ### your code here
            ### set new weights of self.module and call its forward
            self._setweights()
            return self.module.forward(*args)

    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

Протестируйте полученный слой (вновь подставив его в RNNClassifier в качестве rec_layer) с dropout=0.5. Сравните время обучения с предыдущими моделями. Проследите, чтобы качество получилось такое же, как при первой реализации этого дропаута.

In [31]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=FastRNNLayer,
                       dropout=0.5).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 88.250, Testing Loss: 88.349, Training Acc: 0.527, Testing Acc: 0.515
Epoch:   2/ 30 Training Loss: 88.135, Testing Loss: 88.292, Training Acc: 0.533, Testing Acc: 0.522
Epoch:   3/ 30 Training Loss: 87.975, Testing Loss: 88.223, Training Acc: 0.545, Testing Acc: 0.524
Epoch:   4/ 30 Training Loss: 87.784, Testing Loss: 88.133, Training Acc: 0.550, Testing Acc: 0.528
Epoch:   5/ 30 Training Loss: 87.516, Testing Loss: 88.000, Training Acc: 0.559, Testing Acc: 0.533
Epoch:   6/ 30 Training Loss: 87.148, Testing Loss: 87.846, Training Acc: 0.568, Testing Acc: 0.539
Epoch:   7/ 30 Training Loss: 86.666, Testing Loss: 87.623, Training Acc: 0.580, Testing Acc: 0.546
Epoch:   8/ 30 Training Loss: 85.961, Testing Loss: 87.302, Training Acc: 0.593, Testing Acc: 0.554
Epoch:   9/ 30 Training Loss: 85.098, Testing Loss: 86.898, Training Acc: 0.604, Testing Acc: 0.565
Epoch:  10/ 30 Training Loss: 83.855, Testing Loss: 86.247, Training Acc: 0.620, Testing Acc: 0.576


</конец взлома pytorch>

### Реализация дропаута по статье Семениуты и др
Перейдем к реализации дропаута для LSTM по статье [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165). 

Этот метод применения дропаута не менее популярен, чем предыдущий. Его особенность состоит в том, что он придуман специально для гейтовых архитектур. В контексте LSTM этот дропаут накладывается только на информационный поток (m_h - маска дропаута):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
На входы $x_t$ маска накладывается как в предыдущем дропауте. Впрочем, на входы маску можно наложить вообще до вызова рекуррентного слоя.

Согласно статье, маска дропаута может быть как одинаковая, так и разная для всех моментов времени. Мы сделаем одинаковую.

Для реализации этого дропаута можно: а) самостоятельно реализовать LSTM (интерфейса LSTMCell не хватит) б) снова воспользоваться трюком с установкой весов (но тут мы опираемся на свойство hanh(0)=0, к тому же, трюк в данном случае выглядит менее тривиально, чем с дропаутом Гала). Предлагается реализовать дрпоаут по сценираю а. Допишите класс:

In [ ]:
class HandmadeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.):
        super(HandmadeLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.input_weights = nn.Linear(input_size, 4 * hidden_size)
        self.hidden_weights = nn.Linear(hidden_size, 4 * hidden_size)
        
        self.reset_params()


    def reset_params(self):
        """
        initialization as in Pytorch
        do not forget to call this method!
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(0, stdv)
            

    def forward(self, inp, hidden=None):
        ### your code here
        # use functions init_h0_c0 and gen_dropout_masks defined above

Протестируйте вашу реализацию без дропаута (проконтролируйте качество и сравните время обучения с временем обучения nn.LSTM и RNNLayer), а также с dropout=0.5. Сравните качество модели с таким дропаутом с качеством модели с дропаутом Гала и Гарамани.

### Zoneout
Это еще одна модификация идеи дропаута применительно к рекуррентным нейросетям. В Zoneout на каждом временном шаге с вероятностью p компонента скрытого состояния обновляется, а с вероятностью 1-p берется с предыдущего шага. 
В Виде формул (m^t_h - бинарная маска):
 
(сначала обычный рекуррентный переход, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
Затем Zoneout:
$$
h_t = h_t * m_h^t + h_{t-1}*(1-m_h^t)
$$
В этом методе маска уже должна быть разная во все моменты времени (иначе метод упрощается до дропаута Гала и Гарамани). На входы $x_t$ вновь можно накладывать маску до начала работы рекуррентного слоя.  

Если у вас осталось время, вы можете реализовать этот метод. Выберите основу из трех рассмотренных случаев самостоятельно.
